# Causal Inference Pipeline

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dowhy import CausalModel
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
%matplotlib inline

## 1. Load H3 Panel and Weather Data

In [17]:
# Load H3 panel (full)
panel = pd.read_csv('../data/h3_full_panel_res8.csv')
panel['date'] = pd.to_datetime(panel['date']).dt.date

print(f"H3 Panel shape: {panel.shape}")
print(f"Columns: {panel.columns.tolist()}")

# Load weather data
weather = pd.read_csv('../data/nyc_weather_hourly.csv')
weather['date'] = pd.to_datetime(weather['date']).dt.date

print(f"\nWeather shape: {weather.shape}")
print(f"Columns: {weather.columns.tolist()}")

H3 Panel shape: (38871480, 13)
Columns: ['h3_index', 'date', 'hour', 'accidents_count', 'accident_indicator', 'day_of_week', 'is_weekend', 'month', 'is_rush_hour', 'Traffic_Proxy', 'Baseline_Risk', 'rain_flag', 'precipitation']

Weather shape: (34248, 5)
Columns: ['date', 'hour', 'precipitation', 'visibility', 'rain_flag']


## 2. Merge on (Date, Hour)

In [18]:
# Merge panel with weather
print("Merging panel with weather on (date, hour)...")
df = panel.merge(weather[['date', 'hour', 'rain_flag', 'precipitation']], 
                 on=['date', 'hour'], 
                 how='left', suffixes=('', '_w'))

# Ensure we have a rain_flag column (prefer panel's if exists)
if 'rain_flag' not in df.columns and 'rain_flag_w' in df.columns:
    df['rain_flag'] = df['rain_flag_w']
    df.drop(columns=['rain_flag_w'], inplace=True)

print(f"✓ Merge complete")
print(f"  Merged shape: {df.shape}")
print(f"  Columns: {df.columns.tolist()}")
missing_rf = df['rain_flag'].isna().sum() if 'rain_flag' in df.columns else 'NA'
print(f"  Missing rain_flag: {missing_rf}")

# Fill missing rain_flag with 0 (assume no rain if missing)
if 'rain_flag' in df.columns:
    df['rain_flag'] = df['rain_flag'].fillna(0).astype(int)
else:
    raise KeyError("rain_flag not present after merge; check weather file columns")

print(f"\nSample merged rows:")
df.head(10)

Merging panel with weather on (date, hour)...
✓ Merge complete
  Merged shape: (38871480, 15)
  Columns: ['h3_index', 'date', 'hour', 'accidents_count', 'accident_indicator', 'day_of_week', 'is_weekend', 'month', 'is_rush_hour', 'Traffic_Proxy', 'Baseline_Risk', 'rain_flag', 'precipitation', 'rain_flag_w', 'precipitation_w']
  Missing rain_flag: 0

Sample merged rows:


,h3_index,date,hour,accidents_count,accident_indicator,day_of_week,is_weekend,month,is_rush_hour,Traffic_Proxy,Baseline_Risk,rain_flag,precipitation,rain_flag_w,precipitation_w
0,882a100003fffff,2022-01-01,0,0,0,5,1,1,0,1,NaN,0,0.0,0,0.0
1,882a100003fffff,2022-01-01,1,0,0,5,1,1,0,1,0.0,0,0.0,0,0.0
2,882a100003fffff,2022-01-01,2,0,0,5,1,1,0,1,0.0,0,0.0,0,0.0
3,882a100003fffff,2022-01-01,3,0,0,5,1,1,0,1,0.0,0,0.1,0,0.1
4,882a100003fffff,2022-01-01,4,0,0,5,1,1,0,1,0.0,0,0.1,0,0.1
5,882a100003fffff,2022-01-01,5,0,0,5,1,1,0,1,0.0,1,1.1,1,1.1
6,882a100003fffff,2022-01-01,6,0,0,5,1,1,0,1,0.0,1,1.1,1,1.1
7,882a100003fffff,2022-01-01,7,0,0,5,1,1,0,1,0.0,1,0.5,1,0.5
8,882a100003fffff,2022-01-01,8,0,0,5,1,1,0,1,0.0,1,0.7,1,0.7
9,882a100003fffff,2022-01-01,9,0,0,5,1,1,0,1,0.0,1,0.7,1,0.7


## 3. Check Data Quality for Causal Inference

In [19]:
# Check for missing values in key variables
print("Missing values in analysis variables:")
analysis_vars = ['accident_indicator', 'accidents_count', 'rain_flag', 'day_of_week', 'is_weekend', 
                 'month', 'is_rush_hour', 'Baseline_Risk', 'Traffic_Proxy']
print(df[analysis_vars].isnull().sum())

# Drop rows with missing Baseline_Risk or other confounders
df_clean = df[analysis_vars].dropna().copy()

# Sanity: ensure accident_indicator is binary
df_clean['accident_indicator'] = df_clean['accident_indicator'].astype(int)

print(f"\nRows after dropping missing values: {len(df_clean):,} (retained {len(df_clean)/len(df)*100:.2f}%)")

# Class balance
print(f"\nClass balance:")
print(f"  Rain hours: {df_clean['rain_flag'].sum():,} ({df_clean['rain_flag'].mean()*100:.2f}%)")
print(f"  Accident hours (≥1 crash): {df_clean['accident_indicator'].sum():,} ({df_clean['accident_indicator'].mean()*100:.2f}%)")
print(f"  Avg crashes per hour: {df_clean['accidents_count'].mean():.3f}")

# Cross-tabulation: rain vs accident occurrence
print(f"\nCross-tabulation (Rain vs Accident Occurrence):") 
crosstab = pd.crosstab(df_clean['rain_flag'], df_clean['accident_indicator'], 
                       margins=True, normalize='index')
print(crosstab)

print(f"\nNote: Using 'accident_indicator' as outcome (any crash) for interpretability.")

Missing values in analysis variables:
accident_indicator       0
accidents_count          0
rain_flag                0
day_of_week              0
is_weekend               0
month                    0
is_rush_hour             0
Baseline_Risk         1135
Traffic_Proxy            0
dtype: int64

Rows after dropping missing values: 38,870,345 (retained 100.00%)

Class balance:
  Rain hours: 4,182,475 (10.76%)
  Accident hours (≥1 crash): 334,796 (0.86%)
  Avg crashes per hour: 0.009

Cross-tabulation (Rain vs Accident Occurrence):
accident_indicator         0         1
rain_flag                             
0                   0.991481  0.008519
1                   0.990605  0.009395
All                 0.991387  0.008613

Note: Using 'accident_indicator' as outcome (any crash) for interpretability.


Findings:
- Without rain: 0.8516% of hours have crashes
- With rain: 0.9364% of hours have crashes
- Rain is associated with ~11% higher crash probability (0.9364/0.8516 ≈ 1.10)

This is just correlation, not cuasation yet. The 0.0848pp increase is likely due to:
- Confounding: Rain happens at certain times/locations that already have higher crash risk
- Selection bias: Different driver behavior during rain
- Other factors: Visibility, traffic patterns, etc.

## 4. Save Analysis-Ready Dataset

In [5]:
# Save full merged dataset
df.to_csv('../data/analysis_ready.csv', index=False)
print(f"✓ Saved full dataset to ../data/analysis_ready.csv ({df.shape})")

# Save clean dataset for DoWhy
df_clean.to_csv('../data/analysis_ready_clean.csv', index=False)
print(f"✓ Saved clean dataset to ../data/analysis_ready_clean.csv ({df_clean.shape})")
print(f"\nOutcome variable: accident_indicator (1 if ≥1 crash, 0 otherwise)")

✓ Saved full dataset to ../data/analysis_ready.csv ((38871480, 15))
✓ Saved clean dataset to ../data/analysis_ready_clean.csv ((38870345, 9))

Outcome variable: accident_indicator (1 if ≥1 crash, 0 otherwise)


## 5. Define Causal DAG

**Nodes:**
- Treatment: `rain_flag`
- Outcome: `accident_indicator`
- Confounders: `day_of_week`, `is_weekend`, `month`, `is_rush_hour`, `Baseline_Risk`, `Traffic_Proxy`

**Assumptions:**
1. Time features capture all temporal confounding
2. Baseline risk captures location-specific crash propensity
3. Traffic proxy captures exposure variation
4. No unobserved confounders (strong assumption)

In [20]:
# Define DAG
causal_graph = """
digraph {
    day_of_week -> rain_flag;
    day_of_week -> Traffic_Proxy;
    day_of_week -> accident_indicator;
    
    is_weekend -> rain_flag;
    is_weekend -> Traffic_Proxy;
    is_weekend -> accident_indicator;
    
    month -> rain_flag;
    month -> accident_indicator;
    
    is_rush_hour -> Traffic_Proxy;
    is_rush_hour -> accident_indicator;
    
    Baseline_Risk -> accident_indicator;
    Traffic_Proxy -> accident_indicator;
    
    rain_flag -> accident_indicator;
}
"""

print("✓ Causal DAG defined")
print("\nDAG structure:")
print("  Confounders → Rain")
print("  Confounders → Accident Indicator")
print("  Rain → Accident Indicator (causal effect of interest)")

✓ Causal DAG defined

DAG structure:
  Confounders → Rain
  Confounders → Accident Indicator
  Rain → Accident Indicator (causal effect of interest)


## 6. Initialize DoWhy Causal Model

In [21]:
# Initialize causal model
print("Initializing DoWhy causal model...")
print("Outcome: accident_indicator (1 if ≥1 crash in that hour)")

model = CausalModel(
    data=df_clean,
    treatment='rain_flag',
    outcome='accident_indicator',
    graph=causal_graph,
    common_causes=['day_of_week', 'is_weekend', 'month', 'is_rush_hour', 
                   'Baseline_Risk', 'Traffic_Proxy']
 )

print("\n✓ Causal model initialized")
print(f"  Treatment: rain_flag")
print(f"  Outcome: accident_indicator")
print(f"  Confounders: day_of_week, is_weekend, month, is_rush_hour, Baseline_Risk, Traffic_Proxy")

Initializing DoWhy causal model...
Outcome: accident_indicator (1 if ≥1 crash in that hour)

✓ Causal model initialized
  Treatment: rain_flag
  Outcome: accident_indicator
  Confounders: day_of_week, is_weekend, month, is_rush_hour, Baseline_Risk, Traffic_Proxy


## 7. Identify Causal Effect (Backdoor Criterion)

In [22]:
# Identify estimand
print("Identifying causal estimand...")
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

print("\n" + "="*60)
print("IDENTIFIED ESTIMAND")
print("="*60)
print(identified_estimand)

Identifying causal estimand...

IDENTIFIED ESTIMAND
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                                          
────────────(E[accident_indicator|month,day_of_week,is_weekend])
d[rain_flag]                                                    
Estimand assumption 1, Unconfoundedness: If U→{rain_flag} and U→accident_indicator then P(accident_indicator|rain_flag,month,day_of_week,is_weekend,U) = P(accident_indicator|rain_flag,month,day_of_week,is_weekend)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!

### Estimand : 4
Estimand name: general_adjustment
Estimand expression:
     d                                                          
────────────(E[accident_indicator|is_weekend,month,day_of_week])
d[rain_flag]                                                    
Estimand assumption 1, Unconf

## 8.1 Estimate Causal Effect 1 (Propensity Score Matching)

In [23]:
# Estimate using inverse propensity weighting (faster than PSM for large datasets)
print("Estimating causal effect using Propensity Score Weighting...")

estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.propensity_score_weighting",
    target_units="ate"
)

print("\n" + "="*60)
print("CAUSAL EFFECT ESTIMATE (ATE)")
print("="*60)
print(f"Treatment: Rain (rain_flag = 1)")
print(f"Outcome: Accident Indicator (binary)")
print(f"Method: Propensity Score Weighting")

print(f"\nAverage Treatment Effect (ATE): {estimate.value:.6f}")

print(f"\nInterpretation:")
if estimate.value > 0:
    print(f"  Rain INCREASES probability of any crash by {estimate.value*100:.4f} percentage points.")
elif estimate.value < 0:
    print(f"  Rain DECREASES probability of any crash by {abs(estimate.value)*100:.4f} percentage points.")
else:
    print(f"  Rain has NO EFFECT on probability of any crash.")
print("="*60)

Estimating causal effect using Propensity Score Weighting...

CAUSAL EFFECT ESTIMATE (ATE)
Treatment: Rain (rain_flag = 1)
Outcome: Accident Indicator (binary)
Method: Propensity Score Weighting

Average Treatment Effect (ATE): 0.000913

Interpretation:
  Rain INCREASES probability of any crash by 0.0913 percentage points.


## 8.2 Estimate Causal Effect 2 (Linear Regression)

In [32]:

print("Estimating causal effect using Linear Regression...")

estimate_lr = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression",
    target_units="ate",
)

print("\n" + "="*60)
print("CAUSAL EFFECT ESTIMATE (ATE)")
print("="*60)
print(f"Treatment: Rain (rain_flag = 1)")
print(f"Outcome: Accident Indicator (binary)")
print(f"Method: Linear Regression")

print(f"\nAverage Treatment Effect (ATE): {estimate_lr.value:.6f}")

print(f"\nInterpretation:")
if estimate_lr.value > 0:
    print(f"  Rain INCREASES probability of any crash by {estimate_lr.value*100:.4f} percentage points.")
elif estimate_lr.value < 0:
    print(f"  Rain DECREASES probability of any crash by {abs(estimate_lr.value)*100:.4f} percentage points.")
else:
    print(f"  Rain has NO EFFECT on probability of any crash.")
print("="*60)

Estimating causal effect using Linear Regression...

CAUSAL EFFECT ESTIMATE (ATE)
Treatment: Rain (rain_flag = 1)
Outcome: Accident Indicator (binary)
Method: Linear Regression

Average Treatment Effect (ATE): 0.000881

Interpretation:
  Rain INCREASES probability of any crash by 0.0881 percentage points.


## 9.1 Refutation Test 1: Random Common Cause

Add a random confounder. If ATE changes significantly, the estimate is not robust.

In [ ]:
# Refutation: Add random common cause
print("Running refutation test: Random Common Cause...")

refutation_random = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name="random_common_cause"
)

print("\n" + "="*60)
print("REFUTATION: RANDOM COMMON CAUSE")
print("="*60)
print(refutation_random)
print("="*60)

## 9.2 Refutation Test 2: Placebo Treatment

Replace `rain_flag` with a random binary variable. ATE should be ~0.

In [ ]:
# Refutation: Placebo treatment
print("Running refutation test: Placebo Treatment...")

refutation_placebo = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name="placebo_treatment_refuter"
)

print("\n" + "="*60)
print("REFUTATION: PLACEBO TREATMENT")
print("="*60)
print(refutation_placebo)
print("="*60)

## 10. Final Summary

In [ ]:
print("\n" + "#"*60)
print("MVP CAUSAL INFERENCE SUMMARY")
print("#"*60)
print(f"\n🎯 Research Question:")
print(f"   What is the causal effect of rain on accident risk in NYC?")
print(f"\n📊 Data:")
print(f"   - H3 resolution 8 (~600m hexagons)")
print(f"   - {len(df_clean):,} (h3, date, hour) observations")
print(f"   - {df_clean['rain_flag'].sum():,} rain hours ({df_clean['rain_flag'].mean()*100:.2f}%)")
print(f"   - {df_clean['accident_indicator'].sum():,} accident hours ({df_clean['accident_indicator'].mean()*100:.2f}%)")
print(f"\n🔬 Method:")
print(f"   - Causal framework: DoWhy (backdoor adjustment)")
print(f"   - Estimator: Propensity Score Matching")
print(f"   - Confounders: Time features, Baseline Risk, Traffic Proxy")
print(f"\n📈 Result:")
print(f"   - ATE: {estimate.value:.6f}")
if estimate.value > 0:
    print(f"   - Rain INCREASES accident risk by {estimate.value*100:.4f} percentage points")
elif estimate.value < 0:
    print(f"   - Rain DECREASES accident risk by {abs(estimate.value)*100:.4f} percentage points")
else:
    print(f"   - No significant effect detected")
print(f"\n✅ Robustness:")
print(f"   - Random common cause test: {'PASSED' if 'p_value' not in str(refutation_random).lower() or 'fail' not in str(refutation_random).lower() else 'REVIEW'}")
print(f"   - Placebo treatment test: {'PASSED' if 'p_value' not in str(refutation_placebo).lower() or 'fail' not in str(refutation_placebo).lower() else 'REVIEW'}")
print(f"\n🚀 Next Steps:")
print(f"   1. Add more confounders (traffic volume, road conditions)")
print(f"   2. Explore heterogeneous treatment effects by H3 cell")
print(f"   3. Test different rain thresholds (0.1mm vs 1.0mm vs 5.0mm)")
print(f"   4. Extend to other treatments (visibility, temperature)")
print(f"   5. Apply to other cities for external validity")
print("#"*60)
print("\n✓ MVP COMPLETE!")